In [1]:
%pip install azure-cognitiveservices-vision-customvision

Note: you may need to restart the kernel to use updated packages.


In [118]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

endpoint = "https://fimtruscustomvision3.cognitiveservices.azure.com/"
# 문제1
prediction_endpoint = "https://fimtruscustomvision3-prediction.cognitiveservices.azure.com/"
training_key = "855c136ca5484c0d8977a05fab85134e"
prediction_key = "1cfaa22cb1db4a4e8ec3ee3dabd1f0fa"
# 문제2
prediction_resource_id = "/subscriptions/b850d62a-25fe-4d3a-9697-ea40449528a9/resourceGroups/fimtrus-ai-service-group/providers/Microsoft.CognitiveServices/accounts/fimtruscustomvision3-Prediction"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(endpoint, credentials)

prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(prediction_endpoint, prediction_credentials)

print('ttttt')

ttttt


In [43]:
for c in trainer.get_projects():
    print(c.id, c.name)

e7e35ca7-114a-4c2f-b331-0eb2cab19f30 fimtrus-test1111
5672c326-8f9c-493e-8049-2223d7b714f4 fimtrus-customvision3


In [45]:
publish_iteration_name = "fimtrus-kitchen-v1"
project_name = "fimtrus-customvision3"

obj_detection_domain = None  
for domain in trainer.get_domains():  
    if domain.type == "ObjectDetection" and domain.name == "General":  
        obj_detection_domain = domain  
        break

print(obj_detection_domain)
print("create Project!!")

# 현재 존재하는 프로젝트 리스트를 불러온다.
exist_project_list = trainer.get_projects()

is_duplicated = False
duplicated_project_id = ""

for c in exist_project_list:
    # 존재하는 프로젝트 중 내가 지정한 프로젝트명과 동일한 것이 있는지 찾는다.
    if project_name == c.name:
        #만약 같은 이름을 가진 프로젝트가 있다면, True 와 project ID를 저장한다.
        is_duplicated = True
        duplicated_project_id = c.id
        break

# 존재하면 프로젝트 정보를 가져 오고, 없으면 만든다.
if is_duplicated:
    project = trainer.get_project(project_id=duplicated_project_id)
else:
    project = trainer.create_project(project_name, domain_id=obj_detection_domain.id)

print(project)


{'additional_properties': {}, 'id': 'da2e3a8a-40a5-4171-82f4-58522f70fbc1', 'name': 'General', 'type': 'ObjectDetection', 'exportable': False, 'enabled': True, 'exportable_platforms': [], 'model_information': None}
create Project!!
{'additional_properties': {}, 'id': '5672c326-8f9c-493e-8049-2223d7b714f4', 'name': 'fimtrus-customvision3', 'description': '', 'settings': <azure.cognitiveservices.vision.customvision.training.models._models_py3.ProjectSettings object at 0x107f78cd0>, 'created': datetime.datetime(2024, 8, 14, 2, 31, 22, 410000, tzinfo=<isodate.tzinfo.Utc object at 0x106a32d10>), 'last_modified': datetime.datetime(2024, 8, 14, 2, 31, 22, 410000, tzinfo=<isodate.tzinfo.Utc object at 0x106a32d10>), 'thumbnail_uri': 'https://irisprodseatraining.blob.core.windows.net:443/i-5672c3268f9c493e80492223d7b714f4/t-7c8bfd98fd704f309ef102b711441e5e?skoid=a1bb7714-0087-4323-92c2-cfbcc52b1ffc&sktid=33e01921-4d64-4f8c-a055-5bdaffd5e33d&skt=2024-08-14T05%3A51%3A25Z&ske=2024-08-15T05%3A51%3A2

In [14]:
print(project.id, project.name)

5672c326-8f9c-493e-8049-2223d7b714f4 fimtrus-customvision3


In [55]:

exist_tag_list = trainer.get_tags(project_id=project.id)

fork_tag = None
scissors_tag = None

for c in exist_tag_list:
    if c.name == "fork":
        fork_tag = trainer.get_tag(project.id, tag_id=c.id)
    elif c.name == "scissors":
        scissors_tag = trainer.get_tag(project.id, tag_id=c.id)        

if fork_tag is None:
    fork_tag = trainer.create_tag(project.id, "fork")
if scissors_tag is None:
    scissors_tag = trainer.create_tag(project.id, "scissors")

print(fork_tag.name, scissors_tag.name)

fork scissors


In [20]:
fork_image_regions = {
    "fork_1": [ 0.145833328, 0.3509314, 0.5894608, 0.238562092 ],
    "fork_2": [ 0.294117659, 0.216944471, 0.534313738, 0.5980392 ],
    "fork_3": [ 0.09191177, 0.0682516545, 0.757352948, 0.6143791 ],
    "fork_4": [ 0.254901975, 0.185898721, 0.5232843, 0.594771266 ],
    "fork_5": [ 0.2365196, 0.128709182, 0.5845588, 0.71405226 ],
    "fork_6": [ 0.115196079, 0.133611143, 0.676470637, 0.6993464 ],
    "fork_7": [ 0.164215669, 0.31008172, 0.767156839, 0.410130739 ],
    "fork_8": [ 0.118872553, 0.318251669, 0.817401946, 0.225490168 ],
    "fork_9": [ 0.18259804, 0.2136765, 0.6335784, 0.643790841 ],
    "fork_10": [ 0.05269608, 0.282303959, 0.8088235, 0.452614367 ],
    "fork_11": [ 0.05759804, 0.0894935, 0.9007353, 0.3251634 ],
    "fork_12": [ 0.3345588, 0.07315363, 0.375, 0.9150327 ],
    "fork_13": [ 0.269607842, 0.194068655, 0.4093137, 0.6732026 ],
    "fork_14": [ 0.143382356, 0.218578458, 0.7977941, 0.295751631 ],
    "fork_15": [ 0.19240196, 0.0633497, 0.5710784, 0.8398692 ],
    "fork_16": [ 0.140931368, 0.480016381, 0.6838235, 0.240196079 ],
    "fork_17": [ 0.305147052, 0.2512582, 0.4791667, 0.5408496 ],
    "fork_18": [ 0.234068632, 0.445702642, 0.6127451, 0.344771236 ],
    "fork_19": [ 0.219362751, 0.141781077, 0.5919118, 0.6683006 ],
    "fork_20": [ 0.180147052, 0.239820287, 0.6887255, 0.235294119 ]
}

scissors_image_regions = {
    "scissors_1": [ 0.4007353, 0.194068655, 0.259803921, 0.6617647 ],
    "scissors_2": [ 0.426470578, 0.185898721, 0.172794119, 0.5539216 ],
    "scissors_3": [ 0.289215684, 0.259428144, 0.403186262, 0.421568632 ],
    "scissors_4": [ 0.343137264, 0.105833367, 0.332107842, 0.8055556 ],
    "scissors_5": [ 0.3125, 0.09766343, 0.435049027, 0.71405226 ],
    "scissors_6": [ 0.379901975, 0.24308826, 0.32107842, 0.5718954 ],
    "scissors_7": [ 0.341911763, 0.20714055, 0.3137255, 0.6356209 ],
    "scissors_8": [ 0.231617644, 0.08459154, 0.504901946, 0.8480392 ],
    "scissors_9": [ 0.170343131, 0.332957536, 0.767156839, 0.403594762 ],
    "scissors_10": [ 0.204656869, 0.120539248, 0.5245098, 0.743464053 ],
    "scissors_11": [ 0.05514706, 0.159754932, 0.799019635, 0.730392158 ],
    "scissors_12": [ 0.265931368, 0.169558853, 0.5061275, 0.606209159 ],
    "scissors_13": [ 0.241421565, 0.184264734, 0.448529422, 0.6830065 ],
    "scissors_14": [ 0.05759804, 0.05027781, 0.75, 0.882352948 ],
    "scissors_15": [ 0.191176474, 0.169558853, 0.6936275, 0.6748366 ],
    "scissors_16": [ 0.1004902, 0.279036, 0.6911765, 0.477124184 ],
    "scissors_17": [ 0.2720588, 0.131977156, 0.4987745, 0.6911765 ],
    "scissors_18": [ 0.180147052, 0.112369314, 0.6262255, 0.6666667 ],
    "scissors_19": [ 0.333333343, 0.0274019931, 0.443627447, 0.852941155 ],
    "scissors_20": [ 0.158088237, 0.04047389, 0.6691176, 0.843137264 ]
}

## 업로드

In [ ]:
base_image_location = os.path.join("images")
tagged_images_with_regions = []

for file_name in fork_image_regions.keys():
    x, y, w, h = fork_image_regions[file_name] 
    regions = [Region(tag_id=fork_tag.id, left=x, top=y, width=w, height=h)]

    # images/fork/fork_.jpg
    with open(os.path.join(base_image_location, "fork", '{}.jpg'.format(file_name)), 'rb') as image:
        image_data = image.read()
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_data, regions=regions))

for file_name in scissors_image_regions.keys():
    x, y, w, h = scissors_image_regions[file_name]
    regions = [Region(tag_id=scissors_tag.id, left=x, top=y, width=w, height=h)]

    #images/scissors/scissors_.jpg
    with open(os.path.join(base_image_location, "scissors", '{}.jpg'.format(file_name)), "rb") as image:
        image_data = image.read()
        tagged_images_with_regions.append(ImageFileCreateEntry(name=file_name, contents=image_data, regions=regions))

print(len(tagged_images_with_regions))

upload_result = trainer.create_images_from_files(project_id=project.id, batch=ImageFileCreateBatch(images=tagged_images_with_regions))

if upload_result.is_batch_successful:
    print('succeed')
else:
    for image in upload_result.images:
        print("image status: ", image.status)

In [60]:
# Training
iteration = trainer.train_project(project_id=project.id)

while iteration.status != "Completed":
    iteration = trainer.get_iteration(project.id, iteration.id)
    print("Train Status: " + iteration.status)
    time.sleep(3)

print("Completed!")


CustomVisionErrorException: Nothing changed since last training

In [82]:
iteration = trainer.get_iterations(project.id)[0]
print(iteration.name, iteration.status, iteration.id)

Iteration 1 Completed 1e942132-9741-4c87-8486-c213db25be11


In [116]:
# Publish
# trainer.unpublish_iteration(project_id=project.id, iteration_id=iteration.id)
trainer.publish_iteration(project.id, iteration.id, publish_iteration_name, prediction_resource_id)

True

In [120]:
with open(os.path.join(base_image_location, "test", "test_image.jpg"), "rb") as image:
    image_data = image.read()

response = predictor.detect_image(project.id, publish_iteration_name, image_data)
# response = predictor.detect_image(project_id=project.id, published_name=publish_iteration_name, image_data=image_data)

for prediction in response.predictions:
    if prediction.probability > 0.05:
        print('{} : {:0.2f}%'.format(prediction.tag_name, prediction.probability*100))       



fork : 81.24%
